# Ejercicio 1

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class Perceptron:
    def __init__(self, inputs, f_activacion) -> None:
        self.inputs = inputs
        self.f_activacion = f_activacion

        # Inicialización de los pesos
        # Se agrega un elemento para representar el bias
        # y se resta 0.5 para que quede entre -0.5 y 0.5
        self.w = np.random.rand(inputs + 1) - 0.5
    
    def eval(self, patron) :
        x = np.hstack((-1, patron[:self.inputs]))
        # Producto interno
        y = np.inner(x, self.w)
        # No linealidad
        z = self.f_activacion(y)
        # Error
        err = patron[-1] - z
        return (x, z, err)

    def train(self, patron, alpha) :
        # Evaluación
        (x, z, err) = self.eval(patron)
        # Actualización de pesos
        self.w = self.w + (alpha * err) * x
        return (self.w, z, err)

In [4]:
ds_train = np.genfromtxt('icgtp1datos/OR_trn.csv', delimiter=',')
ds_train_size = ds_train.shape[0]

ds_test = np.genfromtxt('icgtp1datos/OR_tst.csv', delimiter=',')
ds_test_size = ds_test.shape[0]

In [5]:
# Función de activación "signo"
def f_sign(a):
    return (np.heaviside(a, 1) * 2) - 1

In [6]:
perceptron = Perceptron(ds_train.shape[1]-1, f_sign)

N = 20          # Nro de épocas
alpha = 1E-4    # Tasa de aprendizaje
nu = 0.3        # Umbral de error

# Para calcular el error se utilizará un conjunto de patrones de tamaño 10%
# del total
N_val = int(np.floor(ds_train_size * 0.1))

# Entrenamiento por epocas
for i in range(N):
    for patron in ds_train: # Para todos los patrones de train
        perceptron.train(patron, alpha)
    
    # Corte del entrenamiento por umbral de error
    errores = 0
    for patron in ds_train[np.random.choice(ds_train_size, size=(N_val), replace=False)]:
        (_, z, _) = perceptron.eval(patron)
        errores += int(z != patron[-1])
    err_med = errores / N_val
    
    # Evolución del error por época
    print(f'Epoca {i}: error medio = {err_med * 100:.2f} %')

    # Si el error calculado es menos del umbral cortar
    if (err_med < nu):
        break

Epoca 0: error medio = 52.00 %
Epoca 1: error medio = 49.00 %
Epoca 2: error medio = 0.00 %


In [7]:
# Test
errores = 0
for patron in ds_test:
    (_, z, _) = perceptron.eval(patron)
    errores += int(z != patron[-1])
err_med = errores / ds_test_size

print(f'Error en test {err_med * 100:.2f} %')

Error en test 0.00 %
